# Introduction

Modules provide a way to run external software from rex. Each module provides a rex function that allows you to pass inputs to the software, and the function will return the outputs to you.

We have adapted the inputs and outputs so that any molecules that the software might want to receive can be passed in `TRC` form, which you can easily extract from `ProteinConformer` and `SmolConformer`. The rest of the configuration for the software can be specified in a `dict`. Each module has documentation, detailing the parameters, outputs, and valid values in the configuration `dict`.

## Common Module Design

The way modules are designed in terms of their inputs and outputs follows a standardized structure.

### Two Forms of Each Module Function: `_rex` and `_rex_s`

The for calling the module is named based on the software it will run followed by `_rex`, e.g. `auto3d_rex`. With this function you have access to all the information provided by the module being run. The module actually returns a `Result`, and in the case of an `err` it will provide a string about what might have gone wrong.

But, there's a simplified version of each module function with the extension `_rex_s` (s for "simplified), e.g. `auto3d_rex_s`. This automatically removes `Result` for you so you get immediate access to the module's actual outputs. Most of the time you'll want to use this, and this is what our examples will use.

### Inputs

* The first argument is the runspec for the module;
* The second argument is the configuration dictionary;
* From there, any additional parameters specific to the module will follow.

### Outputs

* Each module outputs a 

Here's a simple example:

```haskell
{- Returns (Object<Topology>, Object<Residues>, Object<Chains>) -}
auto3d_rex_s default_runspec_gpu { k = 1, optimizing_engine = "ANI2xt" } ["OO"] 
```

* The `default_runspec_gpu` is a built-in shorthand to obtain a runspec that allows us to use one GPU;
* `{ k = 1, optimizing_engine = "ANI2xt" }` is our config dictionary;
*  and `["OO"]` is our module-specific argument, in this case, a list of SMILES strings.

In this case, auto3d is software that will generate a 3D conformation for each SMILES string in a `list` of them. See the auto3d page for more details: here, we're just exhibiting the structure. All modules work in a similar way. For any module that takes a `list` of inputs, it's even smart enough to return the sole output by itself rather than in a `list` with one element, for your convenience!

## Using Objects from Modules

When we obtain an `Object` from a module, we have to do one extra step before we can use it as the input in another module. Technically, the data in that object needs to be repackaged, and there's a rust built-in called `to_data` that does just this. We're working on eliminating the need for this, but for now, make sure to call `to_data` on any `Object` types returned by modules, like so:

```haskell
let
    auto3d = \smi ->  {- write a simplified helper function for the auto3d module -}
        map to_data ( {- map to_data over the tuple of Objects -}
            get 0 (   {- get the first output conformer -}
                auto3d_rex_s default_runspec_gpu { k = 1, optimizing_engine = "ANI2xt" } [smi]
            )
        )
in
    auto3d "OO" {- outputs the smol conformer TR, ready to be passed to another module  -}
```

We can use this `let [...] in [...]` syntax to make helper functions for calling modules that eliminate the repetitive bits. This helper function calls auto3d with a pre-specified runspec and config dict and allows us to easily pass just one SMILES string to the module, automatically "listifying" it and directly returning the sole output conformer.

### `TRCObject`

We've designed all modules to have a separate `Object` for each part of a TRC, so the common type is `(Object<Topology>, Object<Residues>, Object<Chains>)`. This is pretty verbose, so in this documentation we'll refer to this type with the shorthand `TRCObject`.

## Using Modules in your Benchmark

Modules will make up the most of the scripts for running your benchmark. In each module's documentation page, you'll find simple examples of how to call the modules, and in the full benchmark script examples you'll find ways to chain the modules together to do more meaningful tasks.